---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

## Packages

In [102]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.metrics import make_scorer

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler


## Loading data

In [103]:
# load data
train = pd.read_csv('Data/train.csv', encoding = 'cp1252')
test = pd.read_csv('Data/test.csv')
addresses = pd.read_csv('Data/addresses.csv')
latlons = pd.read_csv('Data/latlons.csv')

C:\Users\elmog\AppData\Local\Temp/ipykernel_17504/1581606953.py:2: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv('Data/train.csv', encoding = 'cp1252')


In [104]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250306 entries, 0 to 250305
Data columns (total 34 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   ticket_id                   250306 non-null  int64  
 1   agency_name                 250306 non-null  object 
 2   inspector_name              250306 non-null  object 
 3   violator_name               250272 non-null  object 
 4   violation_street_number     250306 non-null  float64
 5   violation_street_name       250306 non-null  object 
 6   violation_zip_code          0 non-null       float64
 7   mailing_address_str_number  246704 non-null  float64
 8   mailing_address_str_name    250302 non-null  object 
 9   city                        250306 non-null  object 
 10  state                       250213 non-null  object 
 11  zip_code                    250305 non-null  object 
 12  non_us_str_code             3 non-null       object 
 13  country       

In [105]:
# merge latlons and addresses with data
train = pd.merge(train, pd.merge(addresses, latlons, on='address'), on='ticket_id')
test = pd.merge(test, pd.merge(addresses, latlons, on='address'), on='ticket_id')

## Features Analysis

In [106]:
# Checking Nulls
nulls_df = train.isna().sum().reset_index().rename(columns = {0 : 'Null count'})
nulls_df = nulls_df[nulls_df['Null count'] > 0].sort_values(by = 'Null count', ascending=False)
nulls_df = nulls_df.rename(columns = {'Null count': 'Null Percentage'})
nulls_df['Null Percentage'] = nulls_df['Null Percentage'] / train.shape[0]
nulls_df

,index,Null Percentage
6,violation_zip_code,1.000000
31,grafitti_status,0.999996
12,non_us_str_code,0.999988
30,collection_status,0.852592
28,payment_date,0.835749
33,compliance,0.361262
15,hearing_date,0.049903
7,mailing_address_str_number,0.014390
10,state,0.000372
3,violator_name,0.000136


In [107]:
# Features that should be hidden
hidden_info = ['ticket_id', 'agency_name', 'inspector_name', 'violator_name', 'violation_street_number', 'violation_street_name', 'violation_zip_code', 'zip_code', 'mailing_address_str_number', 'non_us_str_code', 'mailing_address_str_name', 'city', 'state', 'country', 'hearing_date', 'ticket_issued_date', 'violation_description', 'address']

# Features to be dropped
features_with_many_nulls = ['grafitti_status']

# Features with rows containing null values
features_with_rows_to_be_dropped = ['compliance']

# Features with rows containing null values that will be imputed
features_to_be_imputed = ['fine_amount', 'lat', 'lon']

# Features that may cause data leakage
columns_cause_data_leakage = ['compliance_detail', 'collection_status']

# Features that are in train data and not found in test data
features_not_in_test = ['payment_amount', 'balance_due', 'payment_status', 'payment_date']

In [108]:
# Drop unnecessary features
train = train.drop(columns=(hidden_info + columns_cause_data_leakage + features_not_in_test + features_with_many_nulls))
test = test.drop(columns = (list(set(hidden_info) - set(['ticket_id'])) + features_with_many_nulls))

# Drop rows with null values
train = train.dropna(axis=0, subset=features_with_rows_to_be_dropped)

# Impute values of 'fine_amount', 'lat', 'lon'
train['fine_amount'] = train['fine_amount'].fillna(train['fine_amount'].median())
train['lat'] = train['lat'].fillna(train['lat'].mean())
train['lon'] = train['lon'].fillna(train['lon'].mean())
test['lat'] = test['lat'].fillna(train['lat'].mean())
test['lon'] = test['lon'].fillna(test['lon'].mean())

In [109]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159880 entries, 0 to 250293
Data columns (total 12 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   violation_code   159880 non-null  object 
 1   disposition      159880 non-null  object 
 2   fine_amount      159880 non-null  float64
 3   admin_fee        159880 non-null  float64
 4   state_fee        159880 non-null  float64
 5   late_fee         159880 non-null  float64
 6   discount_amount  159880 non-null  float64
 7   clean_up_cost    159880 non-null  float64
 8   judgment_amount  159880 non-null  float64
 9   compliance       159880 non-null  float64
 10  lat              159880 non-null  float64
 11  lon              159880 non-null  float64
dtypes: float64(10), object(2)
memory usage: 15.9+ MB


In [110]:
test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 61001 entries, 0 to 61000
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ticket_id        61001 non-null  int64  
 1   violation_code   61001 non-null  object 
 2   disposition      61001 non-null  object 
 3   fine_amount      61001 non-null  float64
 4   admin_fee        61001 non-null  float64
 5   state_fee        61001 non-null  float64
 6   late_fee         61001 non-null  float64
 7   discount_amount  61001 non-null  float64
 8   clean_up_cost    61001 non-null  float64
 9   judgment_amount  61001 non-null  float64
 10  lat              61001 non-null  float64
 11  lon              61001 non-null  float64
dtypes: float64(9), int64(1), object(2)
memory usage: 6.1+ MB


## Encoding categorical data

In [111]:
train['disposition'].unique()

array(['Responsible by Default', 'Responsible by Determination',
       'Responsible by Admission', 'Responsible (Fine Waived) by Deter'],
      dtype=object)

In [112]:
train['violation_code'].unique()

array(['9-1-36(a)', '61-63.0600', '61-63.0500', '61-63.0100',
       '9-1-103(C)', '22-2-83(a)(b)(c)', '22-2-21(b)', '9-1-36(c)',
       '22-2-17', '22-2-23', '61-47.0000/47.0108', '61-90.0100',
       '22-2-88', '19450901', '22-2-43', '9-1-355', '22-2-83', '22-2-22',
       '22-2-45', '9-1-110(a)', '22-2-88(a)', '9-1-104', '9-1-105',
       '22-2-61', '9-1-201(a)', '9-1-209', '61-130.0000/130.0400',
       '22-2-41', '9-1-101', '9-1-43(a) - (Dwellin', '9-1-205', '9-1-206',
       '9-1-107', '9-1-201(b)', '9-1-210', '9-1-220', '20130901',
       '9-1-214', '9-1-103 (a) or (b)', '9-1-477', '9-1-211', '9-1-204',
       '9-1-310', '9-1-208', '9-1-81(a)', '9-1-476', '22-2-44', '9-1-303',
       '22-2-49', '9-1-111', '22-2-16', '9-1-221', '22-2-20', '9-1-36(d)',
       '61-80.0100', '9-1-106', '9-1-108', '9-1-43(a) - (Structu',
       '61-82.0100/32.0066', '61-81.0100/45.0807', '22-2-84(a)(1)(2)(4)',
       '22-2-87(a)', '22-2-56', '19830901', '61-81.0100/32.0066',
       '61-130.0000/130.0

In [113]:
label_encoder = LabelEncoder()
label_encoder.fit(train['disposition'].append(test['disposition'], ignore_index=True))
train['disposition'] = label_encoder.transform(train['disposition'])
test['disposition'] = label_encoder.transform(test['disposition'])
label_encoder.fit(train['violation_code'].append(test['violation_code'], ignore_index=True))
train['violation_code'] = label_encoder.transform(train['violation_code'])
test['violation_code'] = label_encoder.transform(test['violation_code'])

C:\Users\elmog\AppData\Local\Temp/ipykernel_17504/3959255438.py:2: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  label_encoder.fit(train['disposition'].append(test['disposition'], ignore_index=True))
C:\Users\elmog\AppData\Local\Temp/ipykernel_17504/3959255438.py:5: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  label_encoder.fit(train['violation_code'].append(test['violation_code'], ignore_index=True))


## Handeling classes imbalance

In [114]:
# Checking classes imbalance
train['compliance'].value_counts()

0.0    148283
1.0     11597
Name: compliance, dtype: int64

In [115]:
X = train[list(set(train.columns.to_list()) - set(['compliance']))]
y = train['compliance']

In [116]:
# Resampling
# Applying over/under sampling
over = SMOTE(sampling_strategy = {1:50000}, k_neighbors=2, random_state=42)
X_res, y_res = over.fit_resample(X ,y)
under = RandomUnderSampler(sampling_strategy = {0:50000}, random_state=42)
X, y = under.fit_resample(X_res, y_res)

In [117]:
y.value_counts()

0.0    50000
1.0    50000
Name: compliance, dtype: int64

## Train/test splits

In [118]:
# train the model
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=0)

## Training and tuning model parameters

In [52]:
# Grid Search for finding best parameters for random forest classifier

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1000, num = 10)]

# Number of features to consider at every split
max_features = ['auto', 'sqrt']

# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(5, 50, num = 5)]

# Maximum the number of samples to draw from X to train each base estimator
max_samples = [1000, 1250, 1500]

# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'max_samples': max_samples}

scorer = make_scorer(roc_auc_score)

rf = RandomForestClassifier(bootstrap=True, n_jobs=-1, random_state = 2022)

# Instantiate the grid search model
grid_search = GridSearchCV(estimator = rf, 
                           param_grid = random_grid, 
                           scoring = scorer, 
                           cv = 10,
                           n_jobs = -1, 
                           verbose = 2).fit(X, y)

Fitting 10 folds for each of 300 candidates, totalling 3000 fits


E:\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


In [53]:
# Best hyperparameters from the grid search
grid_search.best_params_

{'max_depth': 38,
 'max_features': 'auto',
 'max_samples': 1500,
 'n_estimators': 900}

In [119]:
# Random Forest classifier
rf_clf = RandomForestClassifier(max_depth= 38, 
                                n_estimators= 1000, 
                                random_state=2022,
                                max_samples=1500, 
                                max_features='auto',
                                bootstrap=True, 
                                n_jobs=-1)

rf_clf.fit(X_train, y_train)
rf_y_pred = rf_clf.predict(X_test)
rf_y_pred_prob = rf_clf.predict_proba(X_test)[:, [1]]
print("AUC = ", roc_auc_score(y_test, rf_y_pred))

E:\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features='sqrt'` or remove this parameter as it is also the default value for RandomForestClassifiers and ExtraTreesClassifiers.
  warn(


AUC =  0.759770806253316


## Predicted probabilities for test observations

In [120]:
rf_test_prob = rf_clf.predict_proba(test[list(set(test.columns.to_list()) - set(['ticket_id']))])[:, [1]]

E:\anaconda3\lib\site-packages\sklearn\base.py:493: FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised.
Feature names must be in the same order as they were in fit.

  warnings.warn(message, FutureWarning)


In [121]:
result = pd.DataFrame(rf_test_prob, test.ticket_id).loc[:, 0].rename('compliance')
result

ticket_id
284932    0.473
285362    0.291
285361    0.465
285338    0.508
285346    0.571
          ...  
376496    0.238
376497    0.238
376499    0.537
376500    0.536
369851    0.517
Name: compliance, Length: 61001, dtype: float64

# `Final function to be submitted`

In [124]:
import pandas as pd
import numpy as np

def blight_model():
    
    # Your code here
    from sklearn.preprocessing import LabelEncoder
    from sklearn.ensemble import RandomForestRegressor
    from sklearn.model_selection import train_test_split, GridSearchCV
    from sklearn.metrics import roc_auc_score
    
    # load data
    train = pd.read_csv('Data/train.csv', encoding = 'cp1252')
    test = pd.read_csv('Data/test.csv')
    addresses = pd.read_csv('Data/addresses.csv')
    latlons = pd.read_csv('Data/latlons.csv')
    
    
    # merge latlons and addresses with data
    train = pd.merge(train, pd.merge(addresses, latlons, on='address'), on='ticket_id')
    test = pd.merge(test, pd.merge(addresses, latlons, on='address'), on='ticket_id')
    
    # Feature analysis
    hidden_info = ['ticket_id', 'agency_name', 'inspector_name', 'violator_name', 'violation_street_number', 'violation_street_name', 'violation_zip_code', 'zip_code', 'mailing_address_str_number', 'non_us_str_code', 'mailing_address_str_name', 'city', 'state', 'country', 'hearing_date', 'ticket_issued_date', 'violation_description', 'address']
    features_with_many_nulls = ['grafitti_status']
    columns_cause_data_leakage = ['compliance_detail', 'collection_status']
    features_with_rows_to_be_dropped = ['compliance']
    features_not_in_test = ['payment_amount', 'balance_due', 'payment_status', 'payment_date']
    features_to_be_imputed = ['fine_amount', 'lat', 'lon']
    
    # Handeling Nulls
    train = train.drop((hidden_info + columns_cause_data_leakage + features_not_in_test + features_with_many_nulls), axis=1)
    test = test.drop(columns = (list(set(hidden_info) - set(['ticket_id'])) + features_with_many_nulls))
    train = train.dropna(axis=0, subset=features_with_rows_to_be_dropped)
    
    train['fine_amount'] = train['fine_amount'].fillna(train['fine_amount'].median())
    train['lat'] = train['lat'].fillna(train['lat'].mean())
    train['lon'] = train['lon'].fillna(train['lon'].mean())
    test['lat'] = test['lat'].fillna(train['lat'].mean())
    test['lon'] = test['lon'].fillna(test['lon'].mean())
    
    # Encoding categorical data
    label_encoder = LabelEncoder()
    label_encoder.fit(train['disposition'].append(test['disposition'], ignore_index=True))
    train['disposition'] = label_encoder.transform(train['disposition'])
    test['disposition'] = label_encoder.transform(test['disposition'])
    label_encoder.fit(train['violation_code'].append(test['violation_code'], ignore_index=True))
    train['violation_code'] = label_encoder.transform(train['violation_code'])
    test['violation_code'] = label_encoder.transform(test['violation_code'])
    
    # Splitting features and target
    X = train[list(set(train.columns.to_list()) - set(['compliance']))]
    y = train['compliance']
    
    # Handeling classes imbalance
    from imblearn.over_sampling import SMOTE
    from imblearn.under_sampling import RandomUnderSampler

    over = SMOTE(sampling_strategy = {1:50000}, k_neighbors=2, random_state=42)
    X_res, y_res = over.fit_resample(X ,y)
    under = RandomUnderSampler(sampling_strategy = {0:50000}, random_state=42)
    X, y = under.fit_resample(X_res, y_res)
    
    # Training
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.metrics import f1_score, roc_auc_score

    rf_clf = RandomForestClassifier(n_estimators=900,
                                    max_depth=38,
                                    max_features='auto',
                                    n_jobs=-1, 
                                    max_samples=1500,
                                    bootstrap=True,
                                    random_state = 2022)

    rf_clf.fit(X_train, y_train)
    
    # Predicting probability
    pred_proba = rf_clf.predict_proba(test[list(set(test.columns.to_list()) - set(['ticket_id']))])[:, [1]]
    
    
    
    return pd.DataFrame(pred_proba, test.ticket_id).loc[:, 0].rename('compliance')

In [125]:
blight_model()

C:\Users\elmog\AppData\Local\Temp/ipykernel_17504/2890030489.py:13: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv('Data/train.csv', encoding = 'cp1252')
C:\Users\elmog\AppData\Local\Temp/ipykernel_17504/2890030489.py:44: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  label_encoder.fit(train['disposition'].append(test['disposition'], ignore_index=True))
C:\Users\elmog\AppData\Local\Temp/ipykernel_17504/2890030489.py:47: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  label_encoder.fit(train['violation_code'].append(test['violation_code'], ignore_index=True))
E:\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py:427: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past beh

ticket_id
284932    0.467778
285362    0.283333
285361    0.456667
285338    0.510000
285346    0.566667
            ...   
376496    0.232222
376497    0.232222
376499    0.537778
376500    0.536667
369851    0.513333
Name: compliance, Length: 61001, dtype: float64

### Note
- `imblearn` module may not work on coursera